## Multinomial Naive Bayes
O Multinomial Naive Bayes supõe que os recursos sejam gerados a partir de uma distribuição multinomial simples. A distribuição multinomial descreve a probabilidade de observar contagens entre várias categorias e, portanto, o Multinomial Naive Bayes é mais apropriado para características que representam contagens ou taxas de contagem (variáveis discretas).  

 A distribuição multinomial normalmente requer contagens de entidades inteiras. No entanto, na prática, contagens fracionadas como ``tf-idf`` também podem funcionar.  

O funcionamento é similar ao método gaussiano, exceto que, em vez de modelar a distribuição de dados com a distribuição normal, modelamos a distribuição de dados com uma distribuição multinomial.

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np
import pandas as pd

### Exemplo 1: The 20 newsgroups  
Esse dataset compreende cerca de 18.000 postagens de grupos de notícias sobre 20 tópicos divididos em dois subconjuntos: um para treinamento (ou desenvolvimento) e outro para teste (ou para avaliação de desempenho). A divisão entre o treino e o conjunto de testes é baseada em mensagens postadas antes e depois de uma data específica.

In [2]:
# Importando os dados
from sklearn.datasets import fetch_20newsgroups

In [3]:
# Definindo as categorias 
# Em vez de utilizar as 20 categorias, escolheremos apenas 4, para rodar mais rápido
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [4]:
# Criando o dataset de treino com as categorias selecionadas anteriormente
train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle=True)

len(train.data)  #Verificando a quantidade de linhas do dataset

2257

**Explorando o Dataset**

In [5]:
# Visualizando as classes dos 10 primeiros registros do target
for t in train.target[:10]:
    print(train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [6]:
# O Scikit-Learn registra os labels como array de números, a fim de aumentar a velocidade
train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

**Preprocessamento dos dados**

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVect = TfidfVectorizer(stop_words='english')

In [8]:
# Transformando a coleção de textos (corpus) em uma matriz esparsa
X_train = TfidfVect.fit_transform(train.data)
y_train = train.target

In [9]:
# Verficando as dimensões
print(X_train.shape, y_train.shape)

(2257, 35482) (2257,)


**Treinando o modelo**

In [10]:
MultiNB = MultinomialNB().fit(X_train, y_train)

**Testando o modelo**

In [11]:
# Criando os dados de teste
test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True)

In [12]:
# Convertendo o dataset de teste em matriz esparsa TF-IDF
X_test = TfidfVect.transform(test.data)

In [13]:
# Executando a predição
predicted = MultiNB.predict(X_test)

**Avaliando o modelo**

In [14]:
# Acurácia do Modelo
np.mean(predicted == test.target)

0.8894806924101198

In [15]:
# Métricas
print(metrics.classification_report(test.target, predicted, target_names = test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.72      0.83       319
         comp.graphics       0.95      0.95      0.95       389
               sci.med       0.95      0.88      0.92       396
soc.religion.christian       0.76      0.97      0.85       398

           avg / total       0.90      0.89      0.89      1502



In [16]:
# Confusion Matrix
pd.DataFrame(metrics.confusion_matrix(test.target, predicted),
             index=test.target_names, 
             columns=test.target_names)

,alt.atheism,comp.graphics,sci.med,soc.religion.christian
alt.atheism,231,2,9,77
comp.graphics,3,368,5,13
sci.med,2,13,349,32
soc.religion.christian,3,4,3,388


**Fazendo previsões**

In [17]:
# Criando novos dados
docs_new = ['God is love', 'OpenGL on the GPU is fast', 'Computer brain']

In [18]:
# Transformando em metriz esparsa
x_new = TfidfVect.transform(docs_new)

In [19]:
# Fazendo a predição
predicted = MultiNB.predict(x_new)

In [20]:
#Mostrando os resultados
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
'Computer brain' => sci.med


### Exemplo 2: SMS Spam Collection Dataset
Dataset contendo 5572 mensagens de SMS classificadas como ham (4825) e spam(747).  
https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [21]:
spamData = pd.DataFrame(pd.read_csv('datasets/spam.csv', encoding = "ISO-8859-1"))

**Explorando os dados**

In [22]:
# Verificando as primeiras linhas
spamData.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [23]:
# Verificando se há algum valor nulo no Dataset
spamData.isna().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [24]:
# Verificando a quantidade de ham e spam
spamData['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

**Trabalhando os dados**

In [25]:
# Retirando as colunas desnecessárias
spamData.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis='columns', inplace=True)
spamData.shape

(5572, 2)

In [26]:
# Modificando a coluna target de string para binário
spamData.replace(['ham', 'spam'], [0, 1], inplace=True)

In [27]:
# Verificando o Dataset final
spamData.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
# Dividindo os dataset entre Data e target
X_spam = spamData['v2']
y_spam = spamData['v1']

In [29]:
TfidfVect = TfidfVectorizer(stop_words='english')

# Transformando a coleção de textos (corpus) em uma matriz esparsa
X_spamTf = TfidfVect.fit_transform(X_spam)

In [30]:
# Dividindo entre treino e teste
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_spamTf, y_spam)

**Treinando o modelo**

In [31]:
MultiNB = MultinomialNB().fit(X_train_s, y_train_s)

**Testando o modelo**

In [32]:
pred = MultiNB.predict(X_test_s)

**Avaliando o modelo**

In [33]:
# Acurácia (arredondando para 2 casas decimais)
np.round(np.mean(pred == y_test_s), 2)

0.97

In [34]:
# Métricas
print(metrics.classification_report(y_test_s, pred))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98      1208
          1       1.00      0.77      0.87       185

avg / total       0.97      0.97      0.97      1393



Nosso modelo foi muito bem. Obteve uma acurácia total de 97%, acertando 100% dos casos de SPAM e 96% dos casos de HAM (portanto com 4% de falsos positivos).